## Snippets: Usage of filters and metadata 
https://cloud.google.com/retail/docs/filter-and-order
The expressions for "filter" are expected to work in these examples

In [ ]:
!pip install google.colab

In [ ]:

from google.colab import auth

from google.auth import default
auth.authenticate_user()

creds, _ = default()

In [ ]:
print(creds)

Metadata for the this store <ES_STORE_ID> looks like this:

"id": "1", "structData": {"title": "Document1", "category": ["PersonaA"], "name": "Document1"}, "content": {"mimeType": "application/pdf", "uri": "gs://<BUCKETNAME>/data/Document1"}}


{"id": "2", "structData": {"title": "Document2", "category": ["PersonaA", "PersonaB"], "name": "Document2"}, "content": {"mimeType": "application/pdf", "uri": "gs://<BUCKETNAME>/data/Document2"}}

    

### Below the filter name: ANY(..) matches ensure the query is against only the documents' metadata satisfying that criteria.

In [ ]:
%%bash
curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://discoveryengine.googleapis.com/v1beta/projects/<PROJECT>/locations/global/collections/default_collection/dataStores/<ES_STORE_ID>/servingConfigs/default_search:search" \
-d '{
"servingConfig": "projects/<PROJECT>/locations/global/collections/default_collection/dataStores/<ES_STORE_ID>/servingConfigs/default_search",
"query": "claim",
"filter": "name: ANY(\"Document1\")"
}'

### Below the filter name: ANY(..) matches ensure the query is against only the documents' metadata satisfying that criteria.

In [ ]:
%%bash
curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://discoveryengine.googleapis.com/v1beta/projects/<PROJECT>/locations/global/collections/default_collection/dataStores/<ES_STORE_ID>/servingConfigs/default_search:search" \
-d '{
"servingConfig": "projects/<PROJECT>/locations/global/collections/default_collection/dataStores/<ES_STORE_ID>/servingConfigs/default_search",
"query": "claims",
"filter": "category: ANY(\"PersonaB\")"
}'

### Python Code equivalent

In [ ]:
# Local import of retriever class
import sys, os
#sys.path.append(os.path.abspath(".."))
from utils.es import es_raw_search_summary

In [ ]:
PROJECT_ID = "providersearch-378121"
SEARCH_ENGINE_ID = "bms-eln-07202023_1689884137276"

In [ ]:
query = "how to process a claim"
rawresults = None
summary, rawresults = es_raw_search_summary(PROJECT_ID, SEARCH_ENGINE_ID, query, filtername="name: ANY(\"Document1\")")
print(f"\nQuestion: '{query}'\n\nES Summary: '{summary}'")
print("Rawresult" + "-"*40)
print(rawresults)
print("Summary" + "-"*40)
print(summary)

## In this example I set a slightly more complex filter based on 2 metadata values

In [ ]:
query = "how to process a claim"
rawresults = None
filtersample = "name: ANY(\"Document1\") AND category: ANY(\"PersonaA\")"
summary, rawresults = es_raw_search_summary(PROJECT_ID, SEARCH_ENGINE_ID, query, filtername=filtersample)
print(f"\nQuestion: '{query}'\n\nES Summary: '{summary}'")
prin t("Rawresult" + "-"*40)
print(rawresults)
print("Summary" + "-"*40)
print(summary)

## Create a Control ID for Synonymns

In [48]:
%%bash

curl -X POST \
-H "Authorization: Bearer $(gcloud auth application-default print-access-token)" \
-H "Content-Type: application/json; charset=utf-8" \
-H "X-Goog-User-Project: 9057658179" --data \
'{"displayName": "test2","solutionType": "SOLUTION_TYPE_SEARCH", "useCases": ["SEARCH_USE_CASE_SEARCH"], "synonymsAction": {"synonyms" : ["camzyos","yabagaba345","tororoavds245"]}}' "https://discoveryengine.googleapis.com/v1alpha/projects/9057658179/locations/global/dataStores/bsm-therap_1695917255391/controls?controlId=test2"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   543    0   364  100   179    329    162  0:00:01  0:00:01 --:--:--   491


{
  "name": "projects/9057658179/locations/global/collections/default_collection/dataStores/bsm-therap_1695917255391/controls/test2",
  "displayName": "test2",
  "solutionType": "SOLUTION_TYPE_SEARCH",
  "useCases": [
    "SEARCH_USE_CASE_SEARCH"
  ],
  "synonymsAction": {
    "synonyms": [
      "camzyos",
      "yabagaba345",
      "tororoavds245"
    ]
  }
}


## Update the serving config with the above Control ID 

In [50]:
%%bash
curl -X PATCH -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: 9057658179" \
"https://discoveryengine.googleapis.com/v1alpha/projects/9057658179/locations/global/collections/default_collection/dataStores/bsm-therap_1695917255391/servingConfigs/default_search?update_mask=synonyms_control_ids" \
-d '{"synonymsControlIds": ["test2"]}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   303    0   270  100    33    647     79 --:--:-- --:--:-- --:--:--   726


{
  "name": "projects/9057658179/locations/global/collections/default_collection/dataStores/bsm-therap_1695917255391/servingConfigs/default_search",
  "displayName": "default_search",
  "solutionType": "SOLUTION_TYPE_SEARCH",
  "synonymsControlIds": [
    "test2"
  ]
}


In [ ]:
## Issue a query with the updated servingConfig

In [52]:
%%bash
curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://discoveryengine.googleapis.com/v1beta/projects/9057658179/locations/global/collections/default_collection/dataStores/bsm-therap_1695917255391/servingConfigs/default_search:search" \
-d '{
"servingConfig": "projects/9057658179/locations/global/collections/default_collection/dataStores/bsm-therap_1695917255391/servingConfigs/default_search",
"query": "What are the warnings for yabagaba345",
}'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


{
  "results": [
    {
      "id": "92948d62a9d36e93474bbf860df9e174",
      "document": {
        "name": "projects/9057658179/locations/global/collections/default_collection/dataStores/bsm-therap_1695917255391/branches/0/documents/92948d62a9d36e93474bbf860df9e174",
        "id": "92948d62a9d36e93474bbf860df9e174",
        "derivedStructData": {
          "extractive_answers": [
            {
              "content": "10 6 ADVERSE REACTIONS The following adverse reactions are discussed in greater detail in other sections of the labeling: • cardiac conduction abnormalities [see Warnings and Precautions (5.1)] • rash [see Warnings and Precautions (5.2)] • effects on serum creatinine [see Warnings and Precautions (5.3)] • new onset or worsening renal impairment when used with tenofovir DF [see Warnings and Precautions (5.4)] • chronic kidney disease [see Warnings and Precautions (5.5)] • nephrolithiasis and cholelithiasis [see Warnings and Precautions (5.6)] • hepatotoxicity [see Warning

100 24276    0 24069  100   207  49524    425 --:--:-- --:--:-- --:--:-- 49950


        "name": "projects/9057658179/locations/global/collections/default_collection/dataStores/bsm-therap_1695917255391/branches/0/documents/57f8b311cf5149288a32cdbdb3892acc",
        "id": "57f8b311cf5149288a32cdbdb3892acc",
        "derivedStructData": {
          "link": "gs://rk_bms_therap/pi_onureg.pdf",
          "extractive_answers": [
            {
              "pageNumber": "6",
              "content": "Gastrointestinal Toxicity Advise patients of the risk of gastrointestinal toxicity with ONUREG and of the potential need to use anti-emetic or anti-diarrheal medications during treatment [see Adverse Reactions (6.1)]."
            }
          ]
        }
      }
    },
    {
      "id": "5d0ae82e92294f96c603bb7396fd1618",
      "document": {
        "name": "projects/9057658179/locations/global/collections/default_collection/dataStores/bsm-therap_1695917255391/branches/0/documents/5d0ae82e92294f96c603bb7396fd1618",
        "id": "5d0ae82e92294f96c603bb7396fd1618",
        "d

In [ ]:
%%bash
curl -X PATCH -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: 9057658179" \
-d '{"synonymsControlIds": ["test1"]}' \
"https://discoveryengine.googleapis.com/v1alpha/projects/9057658179/locations/global/collections/default_collection/dataStores/bsm-therap_1695917255391/servingConfigs/default_search" 